In [ ]:
# set module
import pandas as pd

from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#경로 설정
import os

new_dir = "/content/drive/MyDrive/리스크공모전"
os.chdir(new_dir)  # 현재 작업 디렉토리 변경
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium

어제의 농넷 데이터 불러오기

In [ ]:
today = datetime.datetime.today()
yesterday = (today - datetime.timedelta(1)).strftime('%Y%m%d')
url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

response = urllib.request.urlopen(url+yesterday).read()
response = json.loads(response)

data = pd.DataFrame(response['data'])

In [ ]:
data

주산지 선정 및 관측지점 mapping
2022년 1년치 데이터를 기준으로 주산지 선정

In [ ]:
CropWeather_ServiceKey = open('CropWeather_ServiceKey.txt', 'r').read()
year_list = ['2015','2016', '2017', '2018', '2019', '2020']
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
weather = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value

            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0)
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...}
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) :
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])


# DataFrame에 값 채워넣기
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음
error = pd.DataFrame({'year': year_error,
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,
                      '관측시작일': f_obs_date_error})